In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

# CIFAR_10 is a set IMG_CHANNELS = 3 of 60K images 32x32 pixels on Flatten 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

# constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model.add(Dropout(0.25)) 
model.add(Conv2D(64, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(Conv2D(64, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model.add(Dropout(0.25)) 
model.add(Flatten()) 
model.add(Dense(512))
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(NB_CLASSES)) 
model.add(Activation('softmax'))

model.summary()

# compile model
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

# fit model
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

# evaluate model
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

# save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite=True)

# augmenting training set images
NUM_TO_AUGMENT = 5
print("Augmenting training set images...")
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i]  # (3, 32, 32)
    x = x.reshape((1,) + x.shape)  # (1, 3, 32, 32)
    for x_aug in datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='cifar', save_format='jpeg'):
        if num_aug >= NUM_TO_AUGMENT:
            break
        xtas.append(x_aug[0])
        num_aug += 1

# fit the dataget
datagen.fit(X_train)

# train with augmented data
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                              steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
                              epochs=NB_EPOCH, verbose=VERBOSE)

# evaluate model with test data
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])


X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 16, 16, 32)        0         
_______________________________________________

Potenial Ethical and Privacy Concerns
I think the first one that comes to mind is the idea that we could have an AI out there that is more advanced than this one that is constantly watching the public, Which would scare a lot of people if they didn't know something like this was happening another part of this is concerning the data breaches that could happen around an AI like these especially when they are collecting tons of information both in training data and in actual recordings of a person or group of people.

Another part that I think is important to think about is the AIs that use particular artists to train on and then create a new completely AI-generated image but it gets to the point of who owns the trademark of these and how these AI image generation companies could potentially do it the right way in paying the artists for their work and making sure to credit them when posting them online so that way they get a proper reference instead of saying that the AI just came up with it.

Arete. (2023, May 11). The implications of AI for authentic, ethical photography. Medium. https://aretestories.medium.com/the-implications-of-ai-for-authentic-ethical-photography-4022d83dad74 
Gangarapu, K. R. (2022, January 25). Ethics of facial recognition: Key issues and solutions. Learn Hub. https://learn.g2.com/ethics-of-facial-recognition 
Noble, B. (2019, December 11). The ethics of AI Image recognition. Cloudera Blog. https://blog.cloudera.com/the-ethics-of-ai-image-recognition/ 
Roller, J. (2023, December 11). Ethical pros and cons of AI Image Generation. IEEE Computer Society. https://www.computer.org/publications/tech-news/community-voices/ethics-of-ai-image-generation 
